<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC_%EA%B3%B5%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embedding


---


* Word2vec

 > 단어가 가지는 의미 자체를 벡터공간으로 가져오는 것

 > '중심단어'의 '주변단어'들을 이용해 '중심단어'를 추론하는 방식
 
 Word2vec의 장점 : 1. 단어간의 관계 추론이 용이, 2. 단어간의 유사성 측정이 용이 3. 벡터 연산을 통한 추론이 가능(ex 한국 - 서울 + 도쿄 = 일본)

 Word2vec의 단점 : 1.subword information을 무시(서울 vs 서울시 vs 고양시), 2. **Out of Vocabulary(OOV)**에서는 적용 불가능(학습에 사용되지 않은 단어)

* FastText(페이스북 오픈소스)
 
 > Word2vec의 단점을 커버해줌, Word2vec과 사용법은 거의 비슷

 > 한국어는 다양한 용언 형태를 가짐 ==> (동사원형:모르다 , 다른 용언 : 모르네, 모르지, 모르구나...)  

 > 단어의 구성요소를 n_gram으로 나눠 모두 training함 ==> OOV가 나와도 대처 가능

 > **팁**) 워드 임베딩은 음절 단위(대한민국->대한,한민,민국...,n_gram=2-4), 한국어는 자소단위로 나눠서 함(ㄷ,ㅐ,ㅎ,ㅏ,ㄴ...n_gram=6-12) --> 오탈자에 강하다


* Word embedding의 한계점

 > 다의어, 동형어(같은 글자인데 여러가지 뜻을 가지는 단어) 에 대해선 성능이 안좋음

 > 주변 단어를 통해 학습되기 때문에 '문맥'을 고려하지 않음

```
from gensim.models import Word2vec

from gensim.models import FastText
model = FastText(result, size=100, window=5, min_count=5, workers=4, sg=1)
# 학습이 진행되었다면, 이제 electrofishing에 대해서 유사 단어를 찾아보도록 하겠습니다.

model.wv.most_similar("electrofishing")

>>>[('electrolux', 0.7934642434120178), ('electrolyte', 0.78279709815979), ('electro', 0.779127836227417), ('electric', 0.7753111720085144), ('airbus', 0.7648627758026123), ('fukushima', 0.7612422704696655), ('electrochemical', 0.7611693143844604), ('gastric', 0.7483425140380859), ('electroshock', 0.7477173805236816), ('overfishing', 0.7435552477836609)]
```



# Word embedding 실습(Word2vec)

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 30.9MB/s 
     |████████████████████████████████| 92kB 11.4MB/s 
     |████████████████████████████████| 450kB 36.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models import Word2Vec
from konlpy.tag import Okt

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt', filename='ratings.txt')

('ratings.txt', <http.client.HTTPMessage at 0x7fd644f3ecd0>)

In [ ]:
train = pd.read_table('ratings.txt')

In [ ]:
train.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
# 정규표현식으로 한글 외 문자 제거
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")  # ㄱ 에서 ㅎ, ㅏ 에서 ㅣ, 가 에서 힣 외==> 정규 표현식
train = train.dropna()

In [ ]:
train['document'][0]

'어릴때보고 지금다시봐도 재밌어요ㅋㅋ'

In [ ]:
# 불용어 정의(은,는,이,가,...)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '을']

In [ ]:
okt = Okt()
tokenized_data = []
for sentence in train['document']:
    temp_X = okt.morphs(sentence, stem=True)  # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords]  # 불용어 제거
    tokenized_data.append(temp_X)

In [ ]:
tokenized_data[:5]

[['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ'],
 ['디자인',
  '을',
  '배우다',
  '학생',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '을',
  '통해',
  '발전',
  '문화',
  '산업',
  '부럽다',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어렵다',
  '시절',
  '끝',
  '까지',
  '열정',
  '을',
  '지키다',
  '노라노',
  '같다',
  '전통',
  '있다',
  '저',
  '같다',
  '사람',
  '꿈',
  '을',
  '꾸다',
  '이루다',
  '나가다',
  '수',
  '있다',
  '것',
  '감사하다'],
 ['폴리스스토리', '시리즈', '부터', '뉴', '까지', '버리다', '하나', '없다', '최고'],
 ['오다',
  '연기',
  '진짜',
  '개',
  '쩔다',
  '지루하다',
  '생각',
  '몰입',
  '보다',
  '그렇다',
  '이렇다',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱하다', '밤하늘', '뜨다', '있다', '초승달', '같다', '영화']]

In [ ]:
# Word2vec 훈련
model = Word2Vec(sentences=tokenized_data, size=100, window=5, min_count=5, workers=4, sg=0)
model.wv.vectors.shape

(16477, 100)

In [ ]:
# similarities 보기
print(model.wv.most_similar('한석규'))

[('설경구', 0.8939294219017029), ('엄정화', 0.8905223608016968), ('차승원', 0.8891187906265259), ('김명민', 0.8779950737953186), ('안성기', 0.876231849193573), ('최민수', 0.875919759273529), ('박신양', 0.8747932314872742), ('장미희', 0.8695060014724731), ('김수현', 0.8677871227264404), ('전도연', 0.8588352203369141)]


In [ ]:
print(model.wv.most_similar('몰입'))

[('집중', 0.9063402414321899), ('감정이입', 0.7843761444091797), ('긴장', 0.7079639434814453), ('박진', 0.6695482730865479), ('정선', 0.668513298034668), ('긴박', 0.6657693982124329), ('몰입도', 0.6573964357376099), ('이입', 0.644713282585144), ('이해도', 0.6364505887031555), ('빠져들다', 0.6222038865089417)]


# 사전 훈련(Pre-trained)된 Word2Vec 임베딩

위의 예제같이 직접 Word2Vec 을 만들어 사용할 수 있지만, 시간이 오래걸리고 데이터의 양도 한정적이라는 단점이 있기에 위키피디아 등의 방대한 데이터로 사전에 훈련된 워드 임베딩을 가지고 와서 해당 벡터들의 값을 원하는 작업에 사용하는 것도 가능하다.

이러한 사전 훈련된 Word2Vec 을 Pre-trained Word2Vec Embedding 이라고 한다.


---

* 영어

구글에서는 사전 훈련된 3백만 개의 Word2Vec 단어 벡터를 제공한다. 각 임베딩의 차원은 300 이다. gensim을 통해 해당 모델을 불러올 수 있다. 모델을 다운로드 하고 해당 파일 경로에서

모델 다운로드 경로 : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

압축 파일의 용량은 약 1.5GB이지만 압축을 풀면 약 3.3GB의 파일이 나오게된다.


```
import gensim

# 구글의 사전 훈련된 Word2Vec 모델을 로드
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin 파일 경로', binary=True) 
```



---

* 한국어

한국어는 박규병님께서 공개한 Word2Vec 모델이 있다. 해당 깃허브 주소와 모델의 다운로드 링크는 아래와 같다.

GitHub : https://github.com/Kyubyong/wordvectors
모델 다운로드 경로 : https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view

파일의 크기는 약 77MB이고 압축을 풀면 50MV 가량의 ko.bin 파일을 gensim 라이브러리로 로드하면 된다.


```
import gensim
model = gensim.models.Word2Vec.load('ko.bin 파일의 경로')
```



# 언어 모델(Language Model, LM)
 - Markov 확률 기반 언어 모델(마르코프 모델)
  - 마코프 체인 모델
  - 초기의 언어 모델은 다음의 단어나 문장이 나올 확률을 통계와 단어의 n-gram을 기반으로 계산
  - 딥러닝 기반의 언어모델은 해당 확률을 최대로 하도록 네트워크를 학습

 - RNN(Recurrenct Neural Network) 기반 언어 모델
  - 이전 state 정보가 다음 state를 예측하는데 사용됨, 시계열 데이터 처리에 특화

 - seq2seq(Sequence to Sequence)
  - 단점 : 1. sequence가 매우 길 경우 처음에 나온 tokens에 대한 정보가 희박(RNN이다 보니), 2. 중요하지 않은 토큰에 힘을 씀 ==> attention모델의 탄생
  - Encode layer : RNN구조를 통해 context vector를 획득
  - Decode layer : 획득된 context vector를 입력으로 받아서 출력으로 예측

 - attention모델
  - 기존 RNN모델에서 시작
  - 출력된 score에 Softmax를 취함 --> 0-1값을 가짐
  - 해당 값을 가중치(weight)으로 취함 --> dynamic context vector를 획득가능(다른 언어라도 가중치 때문에)
  - RNN모델로 시작하다 보니 순차적으로 연산이 이루어져 느림
 
 - self-attention 


# Bert(bi-directional Transformer)
 - 잘 만들어진 BERT 언어모델 위에 1개의 classification layer만 부착하여 다양한 NLP task를 수행
 - 영어권에서 11개의 NLP에 대해 state-of-the-art 달성
 - from_pretrained, bert-base-encased, bert-large-encased
 - [tokenizing] - 빈도수에 기반해 단어를 의미 있는 패턴으로 잘라서 tokenizing
 - 데이터의 tokenizing
  - Masked language model(MLM) : input token을 일정 확률로 masking
  - Randomly하게 masking:80%, replace:10, unchange:10%
  - Masking 기법 : 숨겨서 학습하게 함
  - Bert 적용 : 감성분석(긍정, 부정), 문장 유사도(이진 분류 기반, 문장 벡터 기반)